<a href="https://colab.research.google.com/github/royseto/langgraph-tutorials-royseto/blob/main/LangSmith_Getting_Started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LangSmith Getting Started
https://docs.smith.langchain.com/

In [ ]:
%%capture --no-stderr
%pip install -U langsmith

In [ ]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Define the following 2 API keys in Google Colab secrets.
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

os.environ["LANGSMITH_PROJECT"] = "langsmith-getting-started"

# The following is documented but not needed in my experiment in Jan 2025.
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [ ]:
import openai
from langsmith import wrappers, traceable

# Auto-trace LLM calls in-context
client = wrappers.wrap_openai(openai.Client())

@traceable # Auto-trace this function
def roys_pipeline(user_input: str):
    result = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="gpt-4o-mini"
    )
    return result.choices[0].message.content

roys_pipeline("Tell me about orangutans!")

"Orangutans are large, tree-dwelling primates found primarily in the rainforests of Borneo and Sumatra. They belong to the family Hominidae, which includes great apes such as gorillas, chimpanzees, and humans. Here are some key points about orangutans:\n\n### Species\nThere are three species of orangutans:\n1. **Bornean orangutan (Pongo pygmaeus)** - Native to Borneo.\n2. **Sumatran orangutan (Pongo abelii)** - Found on the island of Sumatra.\n3. **Tapanuli orangutan (Pongo tapanuliensis)** - A newly recognized species native to a small region in North Sumatra.\n\n### Physical Characteristics\n- **Size**: Adult male orangutans can weigh between 110 to 200 pounds, while females are smaller, typically weighing between 65 to 110 pounds.\n- **Color**: They have long, shaggy red to orange hair, which can sometimes appear brown.\n- **Hands and Feet**: Orangutans have long arms and strong, grasping hands and feet that are adapted for climbing and swinging in trees.\n\n### Behavior and Lifesty

In [ ]:
from langsmith import Client, traceable

client = Client()

# Define dataset: these are your test cases
dataset = client.create_dataset(
    "Sample Dataset 2",
    description="A sample dataset in LangSmith.",
)

client.create_examples(
    inputs=[
        {"postfix": "to LangSmith"},
        {"postfix": "to Evaluations in LangSmith"},
    ],
    outputs=[
        {"response": "Welcome to LangSmith"},
        {"response": "Welcome to Evaluations in LangSmith"},
    ],
    dataset_id=dataset.id,
)

# Define an interface to your application (tracing optional)
@traceable
def dummy_app(inputs: dict) -> dict:
    return {"response": "Welcome " + inputs["postfix"]}

# Define your evaluator(s)
def exact_match(outputs: dict, reference_outputs: dict) -> bool:
    return outputs["response"] == reference_outputs["response"]

# Run the evaluation
experiment_results = client.evaluate(
    dummy_app, # Your AI system goes here
    data=dataset, # The data to predict and grade over
    evaluators=[exact_match], # The evaluators to score the results
    experiment_prefix="sample-experiment", # The name of the experiment
    metadata={"version": "1.0.0", "revision_id": "beta"}, # Metadata about the experiment
    max_concurrency=4,  # Add concurrency.
)

View the evaluation results for experiment: 'sample-experiment-99a9292a' at:
https://smith.langchain.com/o/931abd05-4767-46b4-8d9a-5f4f35abbd31/datasets/a1dbc589-f54d-45cd-8da7-fe38fe8f93bd/compare?selectedSessions=4102b4fb-c206-4a96-94a5-46f7570a9898




0it [00:00, ?it/s]

In [ ]:
%%capture --no-stderr
%pip install pandas

res = experiment_results.to_pandas()

In [ ]:
res

,inputs.postfix,outputs.response,error,reference.response,feedback.exact_match,execution_time,example_id,id
0,to Evaluations in LangSmith,Welcome to Evaluations in LangSmith,None,Welcome to Evaluations in LangSmith,True,0.000846,ee7c53d5-ecf3-45dc-9180-30332bc4c9df,7d81550c-08d8-4e6b-8f83-1c604b3e9937
1,to LangSmith,Welcome to LangSmith,None,Welcome to LangSmith,True,0.000581,306ec95a-ccd2-4fa8-b1aa-500b39805578,1cbcc403-360e-4acf-b75f-dbdc21d316e7
